In [1]:
import pdb;
import scipy.misc as scimisc

from tkinter import *
from PIL import Image
from PIL import ImageTk

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation
from PIL import Image

import MalmoPython
import os
import sys
import time
import random
import json
import numpy as np
import time
from IPython.display import clear_output,display
import logging
import math


import gym
import gym_minecraft
import itertools
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers

import baselines.common.tf_util as U

from baselines import logger
from baselines import deepq
from baselines.deepq.replay_buffer import ReplayBuffer,PrioritizedReplayBuffer
from baselines.common.schedules import LinearSchedule

In [2]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.driving import linear
from bokeh.layouts import row,gridplot
from IPython.display import clear_output,display
import bokeh
output_notebook()

Loading BokehJS ...

In [14]:
from keras import backend as K
from keras.models import Sequential,model_from_json,Model
from keras.layers import Conv2D,LSTM,GRU,TimeDistributed,Dense,Flatten,Input,Lambda
from keras.optimizers import RMSprop,Adam
from keras.callbacks import TensorBoard

In [4]:
import gym
import gym_minecraft
from MinecraftGym import MinecraftWrapper

In [17]:
class Network:
    def __init__(self,stateCnt,actionCnt,batch_size,recurrent,mode,learning_rate):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        self.recurrent = recurrent
        self.mode = mode
        
    def build(self):
        model = Sequential()
        # Lets try a CNN to take screen as input
        # batch size is 64, 320x240 video RGB channels with an extra channel for depth
        #conv1 = 
        model.add(TimeDistributed(Conv2D(64, (3, 3)),
                          input_shape=self.stateCnt,batch_size=self.batch_size))
        
        #model.add(layers.TimeDistributed(layers.Conv2D(32,(8,8),input_shape=self.stateCnt,activation='relu')))
        conv2 = layers.Conv2D(64,(4,4),activation='relu')
        conv3 = layers.Conv2D(64,(3,3),activation='relu')
        model.add(layers.TimeDistributed(conv2))
        model.add(layers.TimeDistributed(conv3))
        model.add(layers.TimeDistributed(layers.Flatten()))
        #model.add(layers.Flatten())
        #model.add(layers.Permute((0,2,1)))
        #self.add(Reshape(input_width, num_filters))
        model.add(layers.GRU(units=70,stateful=True))
        #model.add(layers.Dense(256,activation='relu')
        model.add(layers.Dense(output_dim=self.actionCnt))
        
        model.compile(loss=self._huber_loss,optimizer=Adam(lr=self.learning_rate))
        return model
    def build2(self):
        inpt = Input(shape = self.stateCnt, name = "input")
        
        if self.mode == "linear":
            
            flatten_hidden = Flatten(name = "flatten")(input_data)
            output = Dense(num_actions, name = "output")(flatten_hidden)
        else:
            if self.recurrent:
                # shape should be (timesteps,height,width,color)
                conv1 = TimeDistributed(Conv2D(32, (8, 8), strides = 4, activation = "relu", name = "conv1"))(inpt)
                conv2 = TimeDistributed(Conv2D(64, (4, 4), strides = 2, activation = "relu", name = "conv2"))(conv1)
                conv3 = TimeDistributed(Conv2D(64, (3, 3), strides = 1, activation = "relu", name = "conv3"))(conv2)
                flatten_hidden = TimeDistributed(Flatten())(conv3)
                hidden_input = TimeDistributed(Dense(512, activation = 'relu', name = 'flat_to_512')) (flatten_hidden)
                context = GRU(512, return_sequences=False, stateful=False) (hidden_input)
                
            if self.mode == "dqn":
                h4 = Dense(512, activation='relu', name = "fc")(context)
                output = Dense(num_actions, name = "output")(h4)
            elif self.mode == "duel":
                value_hidden = Dense(512, activation = 'relu', name = 'value_fc')(context)
                value = Dense(1, name = "value")(value_hidden)
                
                action_hidden = Dense(512, activation = 'relu', name = 'action_fc')(context)
                action = Dense(self.actionCnt, name = "action")(action_hidden)
                
                action_mean = Lambda(lambda x: K.mean(x, axis = 1, keepdims = True), name = 'action_mean')(action) 
                output = Lambda(lambda x: x[0] + x[1] - x[2], name = 'output')([action, value, action_mean])
        model = Model(inputs = inpt, outputs = output)
        model.compile(loss=self._huber_loss,optimizer=Adam(lr=self.learning_rate))
        return model
            
        
    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)

In [18]:
test_model = Network((10,80,80,1),6,32,True,'duel',0.001).build2()

In [21]:
test_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 10, 80, 80, 1) 0                                            
____________________________________________________________________________________________________
time_distributed_16 (TimeDistrib (None, 10, 19, 19, 32 2080        input[0][0]                      
____________________________________________________________________________________________________
time_distributed_17 (TimeDistrib (None, 10, 8, 8, 64)  32832       time_distributed_16[0][0]        
____________________________________________________________________________________________________
time_distributed_18 (TimeDistrib (None, 10, 6, 6, 64)  36928       time_distributed_17[0][0]        
___________________________________________________________________________________________

In [24]:
# generate random data of batch size 32
t_data = np.random.random((32,10,80,80,1)) 

t_predict = test_model.predict(t_data)

In [25]:
t_predict.shape

(32, 6)

The Memory will store entire episodes rather than frames from episodes
so instead of a 1D array of frames we now have a 2D array of frames of which the length of 2nd dimension varies.

To sample these experience we will choose a random place in memory and take in a stack of frames (each frame is of a time step) and we will apply this to a CNN+RNN hoping for better results.

In [113]:
class Memory:   # stored as ( s, a, r, s_ )
    def __init__(self, capacity):
        self.samples = []
        self.capacity = capacity
    def add(self, sample):
        self.samples.append(sample)        
        if len(self.samples) > self.capacity:
            self.samples.pop(0)
    def sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)
    def old_sample_episode(self,n,length):
        episodes = self.sample(n)
        episode_samples = []
        for episode in episodes:
            start=None
            sample = None
            if (len(episode)- 1 - length) <= 0 :
                sample = episode
            else:
                start = np.random.randint(0,len(episode)- 1 - length)
                sample = episode[start:(start+length)]
            episode_samples.append(sample)
        return episode_samples
    def sample_episode(self,n,length):
        '''
        return a numpy array of shape n,length,frame_shape
        
        sample time is O(n*length) eh okay. I think there is a better way by vectorizing. Overall this needs to be fixed up
        '''
        episodes = self.sample(n)
        e_s = []
        e_a = []
        e_r = []
        e_s_ = []
        e_done = []
        for episode in episodes:
            s_list = []
            a_list = []
            r_list = []
            s__list = []
            done_list = []
            if (len(episode)- 1 - length) <= 0 :
                sample = episode
            else:
                start = np.random.randint(0,len(episode)- 1 - length)
                sample = episode[start:(start+length)]
            for s,a,r,s_,done in sample:
                s_list.append(s)
                a_list.append(a)
                r_list.append(r)
                s__list.append(s_)
                done_list.append(done)
            
            pad_s = np.zeros((length - len(episode),)+ t[0].shape)
            pad = np.zeros((length - len(episode)))

            #print("pad {}".format(pad_s.shape))
            #print("arr {}".format(np.array(s_list).shape))

            s = np.concatenate((pad_s,np.array(s_list)))
            a = np.concatenate((pad,np.array(a_list)))
            r = np.concatenate((pad,np.array(r_list)))
            s_ = np.concatenate((pad_s,np.array(s__list)))
            done = np.concatenate((pad,np.array(done_list)))
            
            e_s.append(s)
            e_a.append(a)
            e_r.append(r)
            e_s_.append(s_)
            e_done.append(done)
            '''
            smp_s.append(np.concatenate((np.zeros((length - len(s_list))),np.array(s_list))))
            smp_a.append(np.concatenate((np.zeros((length - len(a_list))),np.array(a_list))))
            smp_r.append(np.concatenate((np.zeros((length - len(r_list))),np.array(r_list))))
            smp_s_.append(np.concatenate((np.zeros((length - len(s__list))),np.array(s__list))))
            smp_done.append(np.concatenate((np.zeros((length - len(done_list))),np.array(done_list))))
            np.array(smp_s),np.array(smp_a),np.array(smp_r),np.array(smp_s_),np.array(smp_done)
            '''
            
        return np.array(e_s),np.array(e_a),np.array(e_r),np.array(e_s_),np.array(e_done)

In [114]:
t_mem = Memory(1000)
# generate test data
for e in range(100):
    episode = []
    rand_num = np.random.randint(1,10)
    for frame in range(rand_num):
        s = np.ones((80,80,1))*frame
        a = np.random.randint(0,6)
        r = np.random.randint(0,100)
        s_ = np.ones((80,80,1))*frame+1
        done = False
        if frame == (rand_num-1):
            done = True
        episode.append((s,a,r,s_,done))
    t_mem.add(episode)

In [132]:
ss,sa,sr,ss_,sdone = t_mem.sample_episode(32,10)

In [120]:
res = test_model.predict(ss)

In [121]:
res.shape

(32, 6)

In [108]:
e_s = []
e_a = []
e_r = []
e_s_ = []
e_done = []
for episode in episodes:
    s_list = []
    a_list = []
    r_list = []
    s__list = []
    done_list = []
    #get random location to start
    if (len(episode)- 1 - length) <= 0 :
        sample = episode
    else:
        start = np.random.randint(0,len(episode)- 1 - length)
        sample = episode[start:(start+length)]
    for s,a,r,s_,done in sample:
        s_list.append(s)
        a_list.append(a)
        r_list.append(r)
        s__list.append(s_)
        done_list.append(done)
    pad_s = np.zeros((length - len(episode),)+ t[0].shape)
    pad = np.zeros((length - len(episode)))
    
    #print("pad {}".format(pad_s.shape))
    #print("arr {}".format(np.array(s_list).shape))
    
    s = np.concatenate((pad_s,np.array(s_list)))
    a = np.concatenate((pad,np.array(a_list)))
    r = np.concatenate((pad,np.array(r_list)))
    s_ = np.concatenate((pad_s,np.array(s__list)))
    done = np.concatenate((pad,np.array(done_list)))
    e_s.append(s)
    e_a.append(a)
    e_r.append(r)
    e_s_.append(s_)
    e_done.append(done)

pad (2, 80, 80, 1)
arr (8, 80, 80, 1)
pad (2, 80, 80, 1)
arr (8, 80, 80, 1)
pad (5, 80, 80, 1)
arr (5, 80, 80, 1)
pad (6, 80, 80, 1)
arr (4, 80, 80, 1)
pad (4, 80, 80, 1)
arr (6, 80, 80, 1)
pad (8, 80, 80, 1)
arr (2, 80, 80, 1)
pad (7, 80, 80, 1)
arr (3, 80, 80, 1)
pad (5, 80, 80, 1)
arr (5, 80, 80, 1)
pad (3, 80, 80, 1)
arr (7, 80, 80, 1)
pad (7, 80, 80, 1)
arr (3, 80, 80, 1)
pad (6, 80, 80, 1)
arr (4, 80, 80, 1)
pad (4, 80, 80, 1)
arr (6, 80, 80, 1)
pad (7, 80, 80, 1)
arr (3, 80, 80, 1)
pad (3, 80, 80, 1)
arr (7, 80, 80, 1)
pad (9, 80, 80, 1)
arr (1, 80, 80, 1)
pad (7, 80, 80, 1)
arr (3, 80, 80, 1)
pad (9, 80, 80, 1)
arr (1, 80, 80, 1)
pad (6, 80, 80, 1)
arr (4, 80, 80, 1)
pad (9, 80, 80, 1)
arr (1, 80, 80, 1)
pad (3, 80, 80, 1)
arr (7, 80, 80, 1)
pad (6, 80, 80, 1)
arr (4, 80, 80, 1)
pad (1, 80, 80, 1)
arr (9, 80, 80, 1)
pad (4, 80, 80, 1)
arr (6, 80, 80, 1)
pad (6, 80, 80, 1)
arr (4, 80, 80, 1)
pad (3, 80, 80, 1)
arr (7, 80, 80, 1)
pad (3, 80, 80, 1)
arr (7, 80, 80, 1)
pad (2, 80, 

In [109]:
np.array(e_s).shape

(32, 10, 80, 80, 1)

In [94]:
t_ = np.array(s__list)
t_.shape

(8, 80, 80, 1)

In [91]:
padarr = np.zeros((length - 8,)+t[0].shape)

In [93]:
y = np.concatenate((padarr,t))
y.shape

(10, 80, 80, 1)

In [19]:
class Agent:
    def __init__(self,stateCnt,actionCnt,length,capacity,epsilon,gamma,window_size,learning_rate,name,output_path="./logs"):
        inpt = (window_sz,) + stateCnt
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        self.model = Network(inpt,actionCnt,window_size,learning_rate).build() # model
        self.target_model = Network(inpt,actionCnt,window_size,learning_rate).build() # target model
        self.writer = tf.summary.FileWriter(output_path)
        self.epsilon = epsilon
        self.epsilon_min = 0.01
        self.epsilon_decay = 1/10000
        self.gamma = gamma
        self.steps = 0
        self.memory = Memory(capacity)
        self.name = name
        
        self.loss_count = 0
    def save_model(self,path):
        self.model.save_weights(path)
    def remember(self,sample):
        self.memory.add(sample)
    def save_scalar(self,step, name, value):
        """Save a scalar value to tensorboard.
          Parameters
          ----------
          step: int
            Training step (sets the position on x-axis of tensorboard graph.
          name: str
            Name of variable. Will be the name of the graph in tensorboard.
          value: float
            The value of the variable at this step.
          writer: tf.FileWriter
            The tensorboard FileWriter instance.
          """
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = float(value)
        summary_value.tag = self.name + name
        self.writer.add_summary(summary, step)
    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())
    def act(self, s):
        # Epsilon greedy action selection
        s = s[None][None] # increase the rank of tensor to have a batch_size of 1 and length 1
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.actionCnt)
        act_values = self.model.predict(s)
        return np.argmax(act_values[0]) # returns action
    def replay(self, batch_size,length):
        #print("sample")
        batch = self.memory.old_sample_episode(batch_size,length) # tensor size batch_size,length,frame
        #print(np.array(batch).size)
        for mini in batch:
            #print("reset")
            self.model.reset_states() # we do this because the RNN is stateful
            self.target_model.reset_states()
            
            for s,a,r,s_,done in mini:
                s = s[None][None]
                s_ = s_[None][None]
                #print("predict")
                target = self.model.predict(s)
                if done:
                    target[0][a] = r
                else:
                    a_ = self.model.predict(s_)[0]
                    t = self.target_model.predict(s)[0]
                    target[0][a] = r + self.gamma * t[np.argmax(a_)]
                #print("fit")
                res = self.model.fit(s, target, epochs=1,batch_size=1, verbose=0,shuffle=False)
                self.save_scalar(self.loss_count,"Loss",res.history['loss'][0])
                self.loss_count +=1
        '''     
        for s, a, r, s_, done in minibatch:
            state = state
            next_state = next_state
            
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0,shuffle=False)
        '''

        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay

In [133]:
target = test_model.predict(ss)
a_ = test_model.predict(ss_)

In [134]:
target[:5]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.32822734,  0.07543106,  0.14359573,  0.18191774,  0.04122964,
         0.03736237],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.32822734,  0.07543106,  0.14359573,  0.18191774,  0.04122964,
         0.03736237],
       [ 0.38206726,  0.08710781,  0.15901302,  0.20687872,  0.04828427,
         0.03911546]], dtype=float32)

In [136]:
sa.shape

(32, 10)

# Environment

In [ ]:
def preprocess(rgb_array,scale = 1/12):
    frame_shape = rgb_array.shape
    
    frame = np.array(rgb_array)
    gray_frame = np.dot(frame[...,:3],[0.299,0.587,0.114]).reshape((frame_shape[0],frame_shape[1]))
    smaller = scimisc.imresize(gray_frame,scale,mode='L').astype('float64')
    smaller /= 255.0
    smaller = np.expand_dims(smaller,2) # convert to a 3D array of shape (height,width,grayscale)
    smaller = np.reshape(smaller, [1, *(smaller.shape)])
    return smaller.astype("uint8")

In [ ]:
def render(obs,root,canvas):
    obs = np.squeeze(obs,2)
    image = Image.fromarray(obs.astype('int8'),mode='L')
    photo = ImageTk.PhotoImage(image)
    root.one = photo
    canvas.delete("all")
    canvas.create_image(frame_height,frame_width, image=photo)
    root.update()

## Test

In [ ]:
# Test

root = Tk()
root_frame = Frame(root)
canvas = Canvas(root_frame, borderwidth=0, highlightthickness=0, width=200, height=130, bg="black" )
root_frame.pack()
canvas.pack()

frame_height = 25
frame_width = 35


env = gym.make("MinecraftBasic-v0")
env.load_mission_file("./CliffWalking.xml")
env.init(videoResolution=[420,300],allowContinuousMovement=["move", "turn", "strafe"])


scale = 1/12 # scale image down by 1/12
newshape = (env.video_height*scale,env.video_width*scale,1) # dimension of 1 for grayscale
newshape = tuple(map(int,newshape))

# the pre processor will adjust the observation space therefore we will edit the property of the environment to take the pre processor into accoutn
env.observation_space = gym.spaces.Box(low=0, high=255,
shape=newshape)

done = False

for i in range(1000):
    try:
        env.reset()
        while True:
            action = env.action_space.sample()
            obs, reward, done, info = env.step(action)
            proc_obs = preprocess(obs)
            
            render(proc_obs,root_frame,canvas)

            if done:
                break
    except:
        root.destroy()
        env.close()
        raise
env.close()
root.destroy()


# Plots

In [ ]:
def update(x,y,handle,plot):
    plot.data_source.data['x'] += [x]
    plot.data_source.data['y'] += [y]
    push_notebook(handle=handle)

In [ ]:
inferno = bokeh.palettes.Inferno9
fig1 = figure(plot_width=400, plot_height=400,title="rewards",
                      x_axis_label="x",
                      y_axis_label="y")
rplot = fig1.line([],[],line_width=2)
# make a grid
handle1 = show(fig1, notebook_handle=True)

reward_plot = {"handle":handle1,"plot":rplot}

# Execute

In [11]:
pre_env = gym.make("MinecraftCliffWalking1-v0")
pre_env.init(videoResolution=[400,400],allowContinuousMovement=["move", "turn", "strafe"],observeGrid=[20,-1,20,20,-1,20],observeDistance=[4,45,12])
env = MinecraftWrapper(pre_env,1/5,(41,41))

In [ ]:
atari_env = gym.make('SpaceInvaders-v0')

In [21]:

def train(env,agent,episodes):
    # play loop
    batch_size = 100
    for e in range(episodes):
        R = [0.0]
        agent.model.reset_states()
        s = env.reset()
        #s = preprocess(pre_s,1/5)
        done = False
        e_buffer = []
        for t in itertools.count():
            a = agent.act(s)

            s_, r, done, info = env.step(a)
            #s_ = preprocess(pre_s_,1/5)
            #agent.remember((s,a,r,s_,done))
            e_buffer.append((s,a,r,s_,done))
            s = s_
            R[-1] += r
            if done:
                agent.update_target_model()
                agent.save_scalar(e,"reward per episode",R[-1])
                #print("episode: {}/{}, score: {}, e: {}".format(e, episodes, t, agent.epsilon))
                #exp.metric("reward",R[-1])
                #update(e,R[-1],handle1,rplot)
                R.append(0.0)
                break
        agent.remember(np.array(e_buffer))
        #if e % 100 == 0:
        #    agent.save_model("Exp00-CNN")
        if e % 4 == 0 and e > 1:
            agent.replay(batch_size,20)
    
    

In [22]:
stateCnt = env.observation_space.shape
actionCnt = env.action_space.n
length = 1
mem = 10000
epsilon = 1
gamma =  0.99
window_sz = 1
lr = 0.001
name = "DRQN R3 - GRU100/"
agent = Agent(stateCnt,actionCnt,length,mem,epsilon,gamma,window_sz,lr,name)

/home/ubuntu/miniconda2/envs/casper/lib/python3.5/site-packages/ipykernel/__main__.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6)`


In [ ]:
train(env,agent,1000000)

In [18]:
batch = agent.memory.old_sample_episode(32,10) # tensor size batch_size,length,frame
batch[2].shape


(10, 5)

In [23]:
agent.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (1, 1, 78, 78, 64)        640       
_________________________________________________________________
time_distributed_2 (TimeDist (1, 1, 75, 75, 64)        65600     
_________________________________________________________________
time_distributed_3 (TimeDist (1, 1, 73, 73, 64)        36928     
_________________________________________________________________
time_distributed_4 (TimeDist (1, 1, 341056)            0         
_________________________________________________________________
lstm_1 (LSTM)                (1, 40)                   54575520  
_________________________________________________________________
dense_1 (Dense)              (1, 6)                    246       
Total params: 54,678,934
Trainable params: 54,678,934
Non-trainable params: 0
________________________________________________________________

In [39]:
for mini in batch:
    #print("reset")
    agent.model.reset_states() # we do this because the RNN is stateful
    agent.target_model.reset_states()

    for s,a,r,s_,done in mini:
        s = s[None][None]
        s_ = s_[None][None]
        #print("predict")
        target = agent.model.predict(s)[0]
        #print("target {}".format(target))
        if done:
            target[a] = r
        else:
            a_ = agent.model.predict(s_)[0]
            t = agent.target_model.predict(s)[0]
            target[a] = r + agent.gamma * t[np.argmax(a_)]
        #print("fit")
        '''
        print("target + tderr: {}".format(target))
        print("action: {}".format(a))
        print("argmax next_action: {}".format(np.argmax(a_)))
        print("reward: {}".format(r))
        print("next_action {}".format(a_))
        print("behavioural {}".format(t))
        '''
        res = self.model.fit(s, target, epochs=1,batch_size=1, verbose=0,shuffle=False)
        self.save_scalar(self.loss_count,"Loss",res.history['loss'][0])
        self.loss_count +=1

target [ 0.14771423  0.07921243  0.06676368  0.44944486  0.06276181  0.194103  ]
target + tderr: [ 0.14771423  0.07921243  0.06676368  0.44944486  1.28734303  0.194103  ]
action: 4
argmax next_action: 3
reward: 0.9469377736886901
next_action [ 0.13986205  0.070917    0.0599833   0.4784835   0.05363334  0.19712088]
behavioural [ 0.15228666  0.08425859  0.07051608  0.42550659  0.06783158  0.19960055]
target [ 0.138631    0.06969203  0.0589791   0.48287073  0.05233217  0.19749497]
target + tderr: [ 0.138631    0.06969203  0.0589791   0.48287073  0.05233217  1.30741799]
action: 5
argmax next_action: 3
reward: 0.9470193796151806
next_action [ 0.13846093  0.06952428  0.05884152  0.48347369  0.05215491  0.19754465]
behavioural [ 0.14546727  0.07650387  0.06417615  0.45049819  0.05894516  0.20440933]
target [ 0.13843787  0.06950153  0.05882287  0.48355547  0.0521309   0.19755137]
target + tderr: [ 0.13843787  0.06950153  0.05882287  1.31043613  0.0521309   0.19755137]
action: 3
argmax next_act

target + tderr: [ 0.138631    0.06969203  0.0589791   0.48287073  1.30741799  0.19749497]
action: 4
argmax next_action: 3
reward: 0.9470193796151806
next_action [ 0.13846093  0.06952428  0.05884152  0.48347369  0.05215491  0.19754465]
behavioural [ 0.14546727  0.07650387  0.06417615  0.45049819  0.05894516  0.20440933]
target [ 0.13843787  0.06950153  0.05882287  0.48355547  0.0521309   0.19755137]
target + tderr: [ 1.3108592   0.06950153  0.05882287  0.48355547  0.0521309   0.19755137]
action: 0
argmax next_action: 3
reward: 0.9474353399066398
next_action [ 0.13843472  0.06949845  0.05882033  0.48356655  0.05212763  0.19755226]
behavioural [ 0.14439327  0.07535081  0.06323156  0.4542799   0.05766767  0.20507681]
target [ 0.13843432  0.06949806  0.05882001  0.48356801  0.05212722  0.19755244]
target + tderr: [ 0.13843432  0.06949806  0.05882001  0.48356801  1.31312752  0.19755244]
action: 4
argmax next_action: 3
reward: 0.9492876308296347
next_action [ 0.13843426  0.069498    0.0588199

   5.21271378e-02  -1.00000000e+02]
action: 5
argmax next_action: 3
reward: -100.0
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422148  0.07516794  0.06308171  0.45488125  0.0574661   0.20518151]
target [ 0.14771423  0.07921243  0.06676368  0.44944486  0.06276181  0.194103  ]
target + tderr: [ 1.28734303  0.07921243  0.06676368  0.44944486  0.06276181  0.194103  ]
action: 0
argmax next_action: 3
reward: 0.9469377736886901
next_action [ 0.13986205  0.070917    0.0599833   0.4784835   0.05363334  0.19712088]
behavioural [ 0.15228666  0.08425859  0.07051608  0.42550659  0.06783158  0.19960055]
target [ 0.138631    0.06969203  0.0589791   0.48287073  0.05233217  0.19749497]
target + tderr: [ 0.138631    0.06969203  0.0589791   1.30762768  0.05233217  0.19749497]
action: 3
argmax next_action: 3
reward: 0.947229153439993
next_action [ 0.13846093  0.06952428  0.05884152  0.48347369  0.05215491  0.19754465]
behavioural [ 0.14546727  0.

target [ 0.13843423  0.06949798  0.05881995  0.48356828  0.05212715  0.19755246]
target + tderr: [ 0.13843423  0.06949798  0.05881995  0.48356828  1.29938436  0.19755246]
action: 4
argmax next_action: 3
reward: 0.9354881742133434
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422464  0.07517131  0.06308445  0.45487025  0.05746979  0.20517959]
target [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
target + tderr: [  1.38434231e-01   6.94979727e-02   5.88199347e-02   4.83568281e-01
  -1.00000000e+02   1.97552413e-01]
action: 4
argmax next_action: 3
reward: -100.0
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422464  0.07517131  0.06308445  0.45487025  0.05746979  0.20517959]
target [ 0.14771423  0.07921243  0.06676368  0.44944486  0.06276181  0.194103  ]
target + tderr: [ 0.14771423  0.07921243  0.06676368  0.44944486  0.06276181  1.28734303]
action

argmax next_action: 3
reward: 0.941738677499424
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422193  0.07516839  0.06308208  0.45487982  0.05746659  0.20518126]
target [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
target + tderr: [ 1.29789197  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
action: 0
argmax next_action: 3
reward: 0.9339870993965883
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422153  0.075168    0.06308176  0.45488104  0.05746615  0.20518148]
target [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
target + tderr: [ -1.00000000e+02   6.94979727e-02   5.88199347e-02   4.83568281e-01
   5.21271378e-02   1.97552413e-01]
action: 0
argmax next_action: 3
reward: -100.0
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422153  0.075168    0.063

target + tderr: [ 0.13843787  0.06950153  0.05882287  0.48355547  0.0521309   1.30970943]
action: 5
argmax next_action: 3
reward: 0.9462855217842024
next_action [ 0.13843472  0.06949845  0.05882033  0.48356655  0.05212763  0.19755226]
behavioural [ 0.14439327  0.07535081  0.06323156  0.4542799   0.05766767  0.20507681]
target [ 0.13843432  0.06949806  0.05882001  0.48356801  0.05212722  0.19755244]
target + tderr: [  1.38434321e-01   6.94980621e-02  -1.00000000e+02   4.83568013e-01
   5.21272197e-02   1.97552443e-01]
action: 2
argmax next_action: 3
reward: -100.0
next_action [ 0.13843472  0.06949845  0.05882033  0.48356655  0.05212763  0.19755226]
behavioural [ 0.14439327  0.07535081  0.06323156  0.4542799   0.05766767  0.20507681]
target [ 0.14771423  0.07921243  0.06676368  0.44944486  0.06276181  0.194103  ]
target + tderr: [ 0.14771423  0.07921243  0.06676368  0.44944486  0.06276181  1.28669882]
action: 5
argmax next_action: 3
reward: 0.9462935809594377
next_action [ 0.13986205  0.

argmax next_action: 3
reward: 0.9469377736886901
next_action [ 0.13846093  0.06952428  0.05884152  0.48347369  0.05215491  0.19754465]
behavioural [ 0.14546727  0.07650387  0.06417615  0.45049819  0.05894516  0.20440933]
target [ 0.13843787  0.06950153  0.05882287  0.48355547  0.0521309   0.19755137]
target + tderr: [ 0.13843787  0.06950153  1.30734313  0.48355547  0.0521309   0.19755137]
action: 2
argmax next_action: 3
reward: 0.943919211087426
next_action [ 0.13843472  0.06949845  0.05882033  0.48356655  0.05212763  0.19755226]
behavioural [ 0.14439327  0.07535081  0.06323156  0.4542799   0.05766767  0.20507681]
target [ 0.13843432  0.06949806  0.05882001  0.48356801  0.05212722  0.19755244]
target + tderr: [  1.38434321e-01   6.94980621e-02   5.88200055e-02  -1.00000000e+02
   5.21272197e-02   1.97552443e-01]
action: 3
argmax next_action: 3
reward: -100.0
next_action [ 0.13843472  0.06949845  0.05882033  0.48356655  0.05212763  0.19755226]
behavioural [ 0.14439327  0.07535081  0.063

target + tderr: [ 0.13843423  0.06949798  0.05881995  1.30776119  0.05212715  0.19755246]
action: 3
argmax next_action: 3
reward: 0.9438649533126641
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422464  0.07517131  0.06308445  0.45487025  0.05746979  0.20517959]
target [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
target + tderr: [ 0.13843423  0.06949797  0.05881993  1.30381155  0.05212714  0.19755241]
action: 3
argmax next_action: 3
reward: 0.93990766159151
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422193  0.07516839  0.06308208  0.45487982  0.05746659  0.20518126]
target [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
target + tderr: [  1.38434231e-01   6.94979727e-02   5.88199347e-02   4.83568281e-01
   5.21271378e-02  -1.00000000e+02]
action: 5
argmax next_action: 3
reward: -100.0
next_action [ 0.13843423  0.06

target [ 0.13843432  0.06949806  0.05882001  0.48356801  0.05212722  0.19755244]
target + tderr: [ 0.13843432  0.06949806  0.05882001  0.48356801  1.38553381  0.19755244]
action: 4
argmax next_action: 3
reward: 1.0216939338046545
next_action [ 0.13843426  0.069498    0.05881996  0.48356816  0.05212716  0.19755244]
behavioural [ 0.14424479  0.07519273  0.06310202  0.45479983  0.0574934   0.20516732]
target [ 0.13843423  0.06949798  0.05881995  0.48356828  0.05212715  0.19755246]
target + tderr: [ 0.13843423  0.06949798  0.05881995  0.48356828  0.05212715  1.40284216]
action: 5
argmax next_action: 3
reward: 1.0389459270202823
next_action [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
behavioural [ 0.14422464  0.07517131  0.06308445  0.45487025  0.05746979  0.20517959]
target [ 0.13843423  0.06949797  0.05881993  0.48356828  0.05212714  0.19755241]
target + tderr: [ 0.13843423  0.06949797  0.05881993  0.48356828  1.41722119  0.19755241]
action: 4
argmax next_act

reward: 0.9468528646512652
next_action [ 0.13846093  0.06952428  0.05884152  0.48347369  0.05215491  0.19754465]
behavioural [ 0.14546727  0.07650387  0.06417615  0.45049819  0.05894516  0.20440933]
target [ 0.13843787  0.06950153  0.05882287  0.48355547  0.0521309   0.19755137]
target + tderr: [ 0.13843787  1.3085413   0.05882287  0.48355547  0.0521309   0.19755137]
action: 1
argmax next_action: 3
reward: 0.9451173207209318
next_action [ 0.13843472  0.06949845  0.05882033  0.48356655  0.05212763  0.19755226]
behavioural [ 0.14439327  0.07535081  0.06323156  0.4542799   0.05766767  0.20507681]
target [ 0.13843432  0.06949806  0.05882001  0.48356801  0.05212722  0.19755244]
target + tderr: [  1.38434321e-01  -1.00000000e+02   5.88200055e-02   4.83568013e-01
   5.21272197e-02   1.97552443e-01]
action: 1
argmax next_action: 3
reward: -100.0
next_action [ 0.13843472  0.06949845  0.05882033  0.48356655  0.05212763  0.19755226]
behavioural [ 0.14439327  0.07535081  0.06323156  0.4542799   0.

In [38]:
env.action_space.contains()

TypeError: contains() missing 1 required positional argument: 'x'

# Experiment

In [ ]:
test_buffer = PrioritizedReplayBuffer(50,0.8)

In [ ]:
test_buffer.add(1,2,3,4,8)

In [ ]:
test_buffer.sample(1,0.5)

In [ ]:
dir(test_buffer)

In [ ]:
test_buffer.sample(3,0.9)

In [ ]:
test_buffer.update_priorities([0],[0.99])

In [ ]:
test_buffer.update_priorities

In [ ]:
        test_model = Sequential()
        # Lets try a CNN to take screen as input
        # batch size is 64, 320x240 video RGB channels with an extra channel for depth
        test_model.add(layers.Conv2D(32,(8,8),input_shape=(80,80,1),activation='relu'))
        test_model.add(layers.Conv2D(64,(4,4),activation='relu'))
        
        test_model.add(layers.Conv2D(64,(3,3),activation='relu'))
        test_model.add(layers.Permute((3, 2, 1)))
        test_model.add(layers.Reshape((68, 64)))

In [ ]:
test_model.summary()

In [ ]:
model = Sequential()
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3)),
                          input_shape=(10, 299, 299, 3)))